# Clustering Crypto

In [210]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [211]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('resources/crypto_data.csv', index_col=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [212]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [213]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=['IsTrading'])
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [214]:
# Remove rows that have at least 1 null value.
crypto_df.isnull().sum()

CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [215]:
crypto_df.dropna(inplace=True)
crypto_df


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [216]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] != 0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [217]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name_df = pd.DataFrame(crypto_df['CoinName'], index=crypto_df.index)
coin_name_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [218]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns=['CoinName'])
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [219]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [220]:
X.dtypes

TotalCoinsMined                     float64
TotalCoinSupply                      object
Algorithm_1GB AES Pattern Search      uint8
Algorithm_536                         uint8
Algorithm_Argon2d                     uint8
                                     ...   
ProofType_Proof of Authority          uint8
ProofType_Proof of Trust              uint8
ProofType_TPoS                        uint8
ProofType_Zero-Knowledge Proof        uint8
ProofType_dPoW/PoW                    uint8
Length: 100, dtype: object

In [221]:
X['TotalCoinSupply'] = X['TotalCoinSupply'].astype(float)

In [222]:
X.dtypes

TotalCoinsMined                     float64
TotalCoinSupply                     float64
Algorithm_1GB AES Pattern Search      uint8
Algorithm_536                         uint8
Algorithm_Argon2d                     uint8
                                     ...   
ProofType_Proof of Authority          uint8
ProofType_Proof of Trust              uint8
ProofType_TPoS                        uint8
ProofType_Zero-Knowledge Proof        uint8
ProofType_dPoW/PoW                    uint8
Length: 100, dtype: object

In [223]:
# Standardize the data with StandardScaler().
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,-0.116748,-0.152865,-0.043355,-0.043355,-0.043355,-0.061372,-0.075235,-0.043355,-0.061372,-0.061372,...,1.424222,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355
1,-0.093589,-0.144996,-0.043355,-0.043355,-0.043355,-0.061372,-0.075235,-0.043355,-0.061372,-0.061372,...,1.424222,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355
2,0.525872,4.493764,-0.043355,-0.043355,-0.043355,-0.061372,-0.075235,-0.043355,-0.061372,-0.061372,...,1.424222,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355
3,-0.116354,-0.152554,-0.043355,-0.043355,-0.043355,-0.061372,-0.075235,-0.043355,-0.061372,-0.061372,...,-0.702138,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355
4,-0.114384,-0.152865,-0.043355,-0.043355,-0.043355,-0.061372,-0.075235,-0.043355,-0.061372,-0.061372,...,-0.702138,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,-0.072852,-0.123283,-0.043355,-0.043355,-0.043355,-0.061372,-0.075235,-0.043355,-0.061372,-0.061372,...,-0.702138,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355
529,-0.116420,-0.149167,-0.043355,-0.043355,-0.043355,-0.061372,-0.075235,-0.043355,-0.061372,-0.061372,...,1.424222,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355
530,-0.095234,-0.132154,-0.043355,-0.043355,-0.043355,-0.061372,-0.075235,-0.043355,-0.061372,-0.061372,...,-0.702138,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355
531,-0.116588,-0.152554,-0.043355,-0.043355,-0.043355,-0.061372,-0.075235,-0.043355,-0.061372,-0.061372,...,-0.702138,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355,-0.043355


### Deliverable 2: Reducing Data Dimensions Using PCA

In [224]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)
X_pca

array([[-0.30473302,  0.90568156, -0.63759975],
       [-0.28795361,  0.90528294, -0.63801047],
       [ 2.3387175 ,  1.4819527 , -0.63513837],
       ...,
       [ 0.30582521, -2.25509779,  0.49401212],
       [-0.15655327, -2.05347267,  0.3555344 ],
       [-0.26732113,  0.77170619, -0.30577167]])

In [225]:
# Create a DataFrame with the three principal components.
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'], index=crypto_df.index)
pca_df

,PC1,PC2,PC3
42,-0.304733,0.905682,-0.637600
404,-0.287954,0.905283,-0.638010
1337,2.338718,1.481953,-0.635138
BTC,-0.161177,-1.243325,0.219137
ETH,-0.159523,-2.007268,0.408057
...,...,...,...
ZEPH,2.400060,1.197985,-0.187929
GAP,-0.302765,0.905502,-0.637608
BDX,0.305825,-2.255098,0.494012
ZEN,-0.156553,-2.053473,0.355534


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [226]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [232]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)
len(predictions)

533

In [235]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pca_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = coin_name_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)


(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.304733,0.905682,-0.637600,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.287954,0.905283,-0.638010,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.338718,1.481953,-0.635138,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.161177,-1.243325,0.219137,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.159523,-2.007268,0.408057,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.150653,-1.182443,-0.053906,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.402604,1.225442,-0.496198,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.165898,-2.162979,0.422476,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.157954,-2.007419,0.408052,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.156552,-2.053473,0.355534,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [237]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    data_frame = clustered_df,
    x = 'PC1',
    y = 'PC2',
    z = 'PC3',
    color = 'Class',
    symbol = 'Class'
              )

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [239]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [240]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {len(clustered_df)} tradeable cryptocurrencies")

There are 533 tradeable cryptocurrencies


In [20]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [21]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [22]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)